In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras import metrics
from sklearn.model_selection import train_test_split

In [3]:
import preprocessing
X, y, class_names = preprocessing.create_data_tensor()

/home/parth/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/parth/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/parth/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted a

In [4]:
print('Dimensions of X:', X.shape)
print('Possible Classes:', class_names.values())

Dimensions of X: (2565, 22, 57)
Possible Classes: dict_values(['make', 'polite', 'draw', 'soon', 'money', 'cost', 'when', 'innocent', 'pen', 'name', 'know', 'paper', 'no', 'I', 'tray', 'research', 'computer_PC_', 'ready', 'God', 'what', 'wait_notyet_', 'building', 'yes', 'different', 'sad', 'man', 'right', 'later', 'all', 'hurry', 'his_hers', 'hear', 'danger', 'eat', 'drink', 'share', 'thank', 'you', 'temper', 'juice', 'hurt', 'wild', 'please', 'give', 'come', 'glove', 'forget', 'more', 'which', 'shop', 'lose', 'maybe', 'stubborn', 'question', 'where', 'sorry', 'spend', 'girl', 'Norway', 'write', 'science', 'zero', 'buy', 'happy', 'hot', 'not', 'take', 'will', 'head', 'go', 'is_true_', 'think', 'why', 'deaf', 'answer', 'surprise', 'how', 'read', 'love', 'flash', 'boy', 'voluntary', 'hello', 'cold', 'change_mind_', 'mine', 'crazy', 'responsible', 'who', 'joke', 'same', 'wrong', 'alive', 'us', 'exit'])


In [7]:
X_train_temp, X_test_temp, y_train, y_test = train_test_split(X, y, test_size=0.3)
print('Dimensions of X_train: ', X_train_temp.shape)
print('Dimensions of X_test: ', X_test_temp.shape)

Dimensions of X_train:  (1795, 22, 57)
Dimensions of X_test:  (770, 22, 57)


## Changing the dimensions of the input matrix

In [8]:
X_train = X_train_temp.swapaxes(1, 2)
X_test = X_test_temp.swapaxes(1, 2)
print('Dimensions after swapping the feature axis with the timesteps axis')
print('Dimensions of X_train:', X_train.shape)
print('Dimensions of X_test:', X_test.shape)

Dimensions after swapping the feature axis with the timesteps axis
Dimensions of X_train: (1795, 57, 22)
Dimensions of X_test: (770, 57, 22)


## Two different LSTM models

In [28]:
lstm_model = Sequential()

# Model 1
lstm_model.add(LSTM(units=300, input_shape=(57, 22), return_sequences=True))
lstm_model.add(LSTM(units=300))

# Model 2
# lstm_model.add(LSTM(units=500, input_shape=(57, 22), return_sequences=True))
# lstm_model.add(LSTM(units=500))
# lstm_model.add(Dropout(0.2))

lstm_model.add(Dense(95, activation='tanh'))
lstm_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

## Training

In [31]:
from keras.utils import to_categorical
lstm_model.fit(X_train, to_categorical(y_train), batch_size=16, epochs=10)

Epoch 1/10
1795/1795 [==============================] - 59s 33ms/step - loss: 0.0103 - acc: 0.0407
Epoch 2/10
1795/1795 [==============================] - 61s 34ms/step - loss: 0.0103 - acc: 0.0474
Epoch 3/10
1795/1795 [==============================] - 64s 36ms/step - loss: 0.0103 - acc: 0.0552
Epoch 4/10
1795/1795 [==============================] - 67s 38ms/step - loss: 0.0103 - acc: 0.0546
Epoch 5/10
1795/1795 [==============================] - 68s 38ms/step - loss: 0.0103 - acc: 0.0663
Epoch 6/10
1795/1795 [==============================] - 63s 35ms/step - loss: 0.0102 - acc: 0.0680
Epoch 7/10
1795/1795 [==============================] - 63s 35ms/step - loss: 0.0102 - acc: 0.0691
Epoch 8/10
1795/1795 [==============================] - 63s 35ms/step - loss: 0.0102 - acc: 0.0791
Epoch 9/10
1795/1795 [==============================] - 63s 35ms/step - loss: 0.0101 - acc: 0.0914
Epoch 10/10
1795/1795 [==============================] - 63s 35ms/step - loss: 0.0101 - acc: 0.1097


## Evaluation

In [32]:
score = lstm_model.evaluate(X_test, to_categorical(y_test), batch_size=8)
print(score)

770/770 [==============================] - 9s 11ms/step
[0.010214548137087326, 0.05064935064935065]
